In [9]:
import pandas as pd
from tqdm import tqdm as tqdm
from neo4j import GraphDatabase

In [10]:
df = pd.read_csv("openai-embeddings.csv")

In [11]:
df.head(5)

,movieId,embedding
0,1,"[-0.0271058, -0.0242211, 0.0060390322, -0.0243..."
1,2,"[-0.001596838, -0.022397375, 0.0046575777, 0.0..."
2,3,"[0.008900303, -0.023931928, 0.009981526, -0.02..."
3,4,"[-0.024737105, -0.034573562, -0.0049023638, -0..."
4,5,"[-0.0040508406, -0.024880992, -0.01273487, -0...."


In [12]:
df.iloc[0, 1]

'[-0.0271058, -0.0242211, 0.0060390322, -0.02437703, -0.022557847, -0.0050644707, -0.0134229595, -0.004463491, 0.0017379678, -0.017126292, 0.004986506, -0.008030386, 0.032225497, -0.01233145, 0.011746714, 0.02137538, 0.028535157, -0.00074675767, 0.014007696, -0.01402069, 0.0013668224, 0.007965416, -0.022388924, -0.024831824, 0.0045187166, -0.008329252, 0.023558399, -0.025390573, 0.037189264, 0.0030666199, 0.0086541055, -0.012084561, 0.0061007543, -0.0068544154, -0.032901194, -0.024584936, 0.00486631, -0.028639112, 0.0122015085, -0.005597231, 0.011395872, 0.03752711, -0.029392771, 0.0040866607, -0.00697786, 0.0107331695, 0.017308211, -0.026352141, -0.016047778, 0.00021582475, 0.017048327, 0.029028935, -0.034824327, -0.036409613, -0.008069368, -0.012493878, -0.025312608, 0.0074456492, 0.016476585, -0.007426158, 0.0020888099, 0.008186316, 0.0031559546, 0.003333, -0.013267029, 0.01580089, -0.015670948, -0.0059740613, 0.0025777149, -0.01233145, 0.027365685, 0.02591034, 0.009134889, 0.012838

In [13]:
URI = "neo4j://localhost"
AUTH = ("neo4j", "jabberwocky")

In [14]:
driver = GraphDatabase.driver(URI, auth = AUTH)

In [15]:
driver

In [16]:
records, summary, keys = driver.execute_query("""
    MATCH (m:Movie)
    RETURN ID(m)
""")

In [17]:
element_id_list = [record.data()['ID(m)'] for record in records]

In [19]:
df_filtered = df.loc[element_id_list]

In [20]:
df_filtered.head(5)

,movieId,embedding
0,1,"[-0.0271058, -0.0242211, 0.0060390322, -0.0243..."
9,10,"[-0.02352419, -0.03164042, -0.009597766, -0.02..."
10,11,"[-0.011790864, -0.024745937, -0.0066942098, -0..."
11,12,"[-0.021887923, -0.034233358, 0.00079183327, -0..."
15,16,"[-0.0054488634, -0.02576181, 0.021457337, -0.0..."


In [21]:
for index, row in tqdm(df_filtered.iterrows(), total = len(df_filtered)):
    movieId = index
    embedding = row["embedding"]
    query = """
        MATCH (m:Movie)
        WHERE ID(m) = {}
        SET m.embedding = {}
    """.format(movieId, embedding)
    driver.execute_query(query)

100%|██████████| 38/38 [01:24<00:00,  2.22s/it]


In [22]:
records, summary, keys = driver.execute_query("""
    MATCH (m:Movie)
    WHERE ID(m) = 0
    RETURN m
""")

In [23]:
for result in records:
    print(result)

<Record m=<Node element_id='4:e46b6252-efcd-4468-b0de-20e7f42a6a2b:0' labels=frozenset({'Movie'}) properties={'tagline': 'Welcome to the Real World', 'embedding': [-0.0271058, -0.0242211, 0.0060390322, -0.02437703, -0.022557847, -0.0050644707, -0.0134229595, -0.004463491, 0.0017379678, -0.017126292, 0.004986506, -0.008030386, 0.032225497, -0.01233145, 0.011746714, 0.02137538, 0.028535157, -0.00074675767, 0.014007696, -0.01402069, 0.0013668224, 0.007965416, -0.022388924, -0.024831824, 0.0045187166, -0.008329252, 0.023558399, -0.025390573, 0.037189264, 0.0030666199, 0.0086541055, -0.012084561, 0.0061007543, -0.0068544154, -0.032901194, -0.024584936, 0.00486631, -0.028639112, 0.0122015085, -0.005597231, 0.011395872, 0.03752711, -0.029392771, 0.0040866607, -0.00697786, 0.0107331695, 0.017308211, -0.026352141, -0.016047778, 0.00021582475, 0.017048327, 0.029028935, -0.034824327, -0.036409613, -0.008069368, -0.012493878, -0.025312608, 0.0074456492, 0.016476585, -0.007426158, 0.0020888099, 0.0

In [27]:
driver.execute_query("""
    CALL db.index.vector.createNodeIndex(
        'moviePlots',
        'Movie',
        'embedding',
        1536,
        'cosine'
    )
""")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x000002330C6A4890>, keys=[])